# Lane detection

In this notebook the lane detection was build adn tested

In [1]:
import numpy as np
import cv2
import glob
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
%matplotlib qt
from pathlib import Path
import os
import pipeline as pp

In [18]:
# Make a list of calibration images
images = glob.glob('../test_images/*.jpg')

def plotimage(img, i):
    plt.figure(i)
    plt.imshow(img)

# select images between 0 and 7
fname = images[7]

# original
img = mpimg.imread(fname)
plotimage(img, 0)

# undistorted
img_undist = pp.undistort_image(img)
plotimage(img_undist, 1)

# bitmap with streigh guides
bitmap = pp.threshold(img_undist)
# Stack each channel
bitmap_to_plot = np.dstack(( bitmap, bitmap, bitmap)) * 255
pts = np.array(
    [[293, 670],
     [577, 463],
     [704, 463],
     [1018, 670]], np.int32)
pts = pts.reshape((-1,1,2))
cv2.polylines(bitmap_to_plot,[pts],True,(0,255,255), 2)

plotimage(bitmap_to_plot, 2)

# eagle view with streigh guides
img_eagle_eye = pp.eagle_eye(bitmap_to_plot)

x_half = int(img_eagle_eye.shape[0] / 2)
y_half = int(img_eagle_eye.shape[1] / 2)
bottom_half = img_eagle_eye[x_half:, :]

plotimage(bottom_half, 3)

To compute the first polinomial fit I will use moving windows. Below we can my implementation

In [3]:
def find_lane_pixels(binary_warped, nwindows, margin, minpix):
    # Take a histogram of the bottom half of the image
    histogram = np.sum(binary_warped[binary_warped.shape[0]//2:,:], axis=0)
    # Create an output image to draw on and visualize the result
    out_img = np.dstack((binary_warped, binary_warped, binary_warped))
    # Find the peak of the left and right halves of the histogram
    # These will be the starting point for the left and right lines
    midpoint = np.int(histogram.shape[0]//2)
    leftx_base = np.argmax(histogram[:midpoint])
    rightx_base = np.argmax(histogram[midpoint:]) + midpoint

    # Set height of windows - based on nwindows above and image shape
    window_height = np.int(binary_warped.shape[0]//nwindows)
    # Identify the x and y positions of all nonzero pixels in the image
    nonzero = binary_warped.nonzero()
    nonzeroy = np.array(nonzero[0])
    nonzerox = np.array(nonzero[1])
    # Current positions to be updated later for each window in nwindows
    leftx_current = leftx_base
    rightx_current = rightx_base

    # Create empty lists to receive left and right lane pixel indices
    left_lane_inds = []
    right_lane_inds = []

    # Step through the windows one by one
    for window in range(nwindows):
        # Identify window boundaries in x and y (and right and left)
        win_y_low = binary_warped.shape[0] - (window+1)*window_height
        win_y_high = binary_warped.shape[0] - window*window_height
        ### TO-DO: Find the four below boundaries of the window ###
        win_xleft_low = leftx_current - margin # Update this
        win_xleft_high = leftx_current + margin # Update this
        win_xright_low = rightx_current - margin # Update this
        win_xright_high = rightx_current + margin # Update this
        
        # Draw the windows on the visualization image
        cv2.rectangle(out_img,(win_xleft_low,win_y_low),
        (win_xleft_high,win_y_high),(0,255,0), 2) 
        cv2.rectangle(out_img,(win_xright_low,win_y_low),
        (win_xright_high,win_y_high),(0,255,0), 2) 
        
        ### TO-DO: Identify the nonzero pixels in x and y within the window ###
        good_left_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & 
        (nonzerox >= win_xleft_low) &  (nonzerox < win_xleft_high)).nonzero()[0]
        good_right_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & 
        (nonzerox >= win_xright_low) &  (nonzerox < win_xright_high)).nonzero()[0]
        
        # Append these indices to the lists
        left_lane_inds.append(good_left_inds)
        right_lane_inds.append(good_right_inds)
        
        ### TO-DO: If you found > minpix pixels, recenter next window ###
        if minpix < len(good_left_inds):
            leftx_current = np.int(np.mean(nonzerox[good_left_inds]))
        
        if minpix < len(good_right_inds):
            rightx_current = np.int(np.mean(nonzerox[good_right_inds])) 

    # Concatenate the arrays of indices (previously was a list of lists of pixels)
    try:
        left_lane_inds = np.concatenate(left_lane_inds)
        right_lane_inds = np.concatenate(right_lane_inds)
    except ValueError:
        # Avoids an error if the above is not implemented fully
        pass

    # Extract left and right line pixel positions
    leftx = nonzerox[left_lane_inds]
    lefty = nonzeroy[left_lane_inds] 
    rightx = nonzerox[right_lane_inds]
    righty = nonzeroy[right_lane_inds]

    return leftx, lefty, rightx, righty, out_img


def fit_polynomial(binary_warped, nwindows, margin, minpix):
    # Find our lane pixels first
    leftx, lefty, rightx, righty, out_img = find_lane_pixels(binary_warped, nwindows, margin, minpix)

    # Fit a second order polynomial to each using `np.polyfit` ###
    left_fit = np.polyfit(lefty, leftx, 2)
    right_fit = np.polyfit(righty, rightx, 2)

    # Generate x and y values for plotting
    ploty = np.linspace(0, binary_warped.shape[0]-1, binary_warped.shape[0] )
    try:
        left_fitx = left_fit[0]*ploty**2 + left_fit[1]*ploty + left_fit[2]
        right_fitx = right_fit[0]*ploty**2 + right_fit[1]*ploty + right_fit[2]
    except TypeError:
        # Avoids an error if `left` and `right_fit` are still none or incorrect
        print('The function failed to fit a line!')
        left_fitx = 1*ploty**2 + 1*ploty
        right_fitx = 1*ploty**2 + 1*ploty

    ## Visualization ##
    # Colors in the left and right lane regions
    out_img[lefty, leftx] = [255, 0, 0]
    out_img[righty, rightx] = [0, 0, 255]

    # Plots the left and right polynomials on the lane lines
    plt.plot(left_fitx, ploty, color='yellow')
    plt.plot(right_fitx, ploty, color='yellow')

    return out_img, left_fit, right_fit

# Making output directory
output_dir = "../output_images/lane_detection/moving_window/"
Path(output_dir).mkdir(parents=True, exist_ok=True)

i = 0

# list to save the fitting of the images
left_fit_list = []
right_fit_list = []

for fname in images:
    # original
    img = mpimg.imread(fname)
    # undistorted
    img_undist = pp.undistort_image(img)
    # bitmap with streigh guides
    bitmap = pp.threshold(img_undist, s_thresh=(150, 255), sx_thresh=(20, 150))
    # eagle view with streigh guides
    img_eagle_eye = pp.eagle_eye(bitmap)

    # taking half to the image 
    x_half = int(img_eagle_eye.shape[0] / 2)
    y_half = int(img_eagle_eye.shape[1] / 2)
    bottom_half = img_eagle_eye[x_half:, :]

    f = plt.figure(i)
    plt.clf()
    i = i + 1

    out_img, left_fit, right_fit = fit_polynomial(bottom_half, 10, 100, 40)
    plt.imshow(out_img)
    f.savefig(output_dir + os.path.basename(fname))

    left_fit_list.append(left_fit)
    right_fit_list.append(right_fit)

As we can observe the images were correctly fitted for example:

![Lanes Image](../output_images/lane_detection/moving_window/test2.jpg) 

Using the moving window I plan to obtain the first fit. After I will use the fitting of the previous image to delimite the area of interest

In [19]:
def fit_poly(img_shape, leftx, lefty, rightx, righty):
    # Fit a second order polynomial to each with np.polyfit() ###
    left_fit = np.polyfit(lefty, leftx, 2)
    right_fit = np.polyfit(righty, rightx, 2)
    # Generate x and y values for plotting
    ploty = np.linspace(0, img_shape[0]-1, img_shape[0])
    ### TO-DO: Calc both polynomials using ploty, left_fit and right_fit ###
    left_fitx = left_fit[0]*ploty**2 + left_fit[1]*ploty + left_fit[2]
    right_fitx = right_fit[0]*ploty**2 + right_fit[1]*ploty + right_fit[2]
    
    return left_fitx, right_fitx, ploty

def search_around_poly(binary_warped, left_fit, right_fit):
    # HYPERPARAMETER
    # Choose the width of the margin around the previous polynomial to search
    # The quiz grader expects 100 here, but feel free to tune on your own!
    margin = 100

    # Grab activated pixels
    nonzero = binary_warped.nonzero()
    nonzeroy = np.array(nonzero[0])
    nonzerox = np.array(nonzero[1])
    
    ### TO-DO: Set the area of search based on activated x-values ###
    ### within the +/- margin of our polynomial function ###
    ### Hint: consider the window areas for the similarly named variables ###
    ### in the previous quiz, but change the windows to our new search area ###
    left_lane_inds = ((nonzerox - (left_fit[0]*nonzeroy**2 + left_fit[1]*nonzeroy + left_fit[2]))**2 < margin**2).nonzero()[0]
    right_lane_inds = ((nonzerox - (right_fit[0]*nonzeroy**2 + right_fit[1]*nonzeroy + right_fit[2]))**2 < margin**2).nonzero()[0]
    
    # Again, extract left and right line pixel positions
    leftx = nonzerox[left_lane_inds]
    lefty = nonzeroy[left_lane_inds] 
    rightx = nonzerox[right_lane_inds]
    righty = nonzeroy[right_lane_inds]

    # Fit new polynomials
    left_fitx, right_fitx, ploty = fit_poly(binary_warped.shape, leftx, lefty, rightx, righty)
    
    ## Visualization ##
    # Create an image to draw on and an image to show the selection window
    out_img = np.dstack((binary_warped, binary_warped, binary_warped))*255
    window_img = np.zeros_like(out_img)
    # Color in left and right line pixels
    out_img[nonzeroy[left_lane_inds], nonzerox[left_lane_inds]] = [255, 0, 0]
    out_img[nonzeroy[right_lane_inds], nonzerox[right_lane_inds]] = [0, 0, 255]

    # Generate a polygon to illustrate the search window area
    # And recast the x and y points into usable format for cv2.fillPoly()
    left_line_window1 = np.array([np.transpose(np.vstack([left_fitx-margin, ploty]))])
    left_line_window2 = np.array([np.flipud(np.transpose(np.vstack([left_fitx+margin, 
                              ploty])))])
    left_line_pts = np.hstack((left_line_window1, left_line_window2))
    right_line_window1 = np.array([np.transpose(np.vstack([right_fitx-margin, ploty]))])
    right_line_window2 = np.array([np.flipud(np.transpose(np.vstack([right_fitx+margin, 
                              ploty])))])
    right_line_pts = np.hstack((right_line_window1, right_line_window2))

    # Draw the lane onto the warped blank image
    cv2.fillPoly(window_img, np.int_([left_line_pts]), (0,255, 0))
    cv2.fillPoly(window_img, np.int_([right_line_pts]), (0,255, 0))
    result = cv2.addWeighted(out_img, 1, window_img, 0.3, 0)
    
    # Plot the polynomial lines onto the image
    plt.plot(left_fitx, ploty, color='yellow')
    plt.plot(right_fitx, ploty, color='yellow')
    ## End visualization steps ##
    
    return result

# Making output directory
output_dir = "../output_images/lane_detection/poly_filter/"
Path(output_dir).mkdir(parents=True, exist_ok=True)

for i in range(len(left_fit_list)):
    # original
    img = mpimg.imread(images[i])
    # undistorted
    img_undist = pp.undistort_image(img)
    # bitmap with streigh guides
    bitmap = pp.threshold(img_undist, s_thresh=(150, 255), sx_thresh=(20, 150))
    # eagle view with streigh guides
    img_eagle_eye = pp.eagle_eye(bitmap)

    # taking half to the image 
    x_half = int(img_eagle_eye.shape[0] / 2)
    y_half = int(img_eagle_eye.shape[1] / 2)
    bottom_half = img_eagle_eye[x_half:, :]

    left_fit  = left_fit_list[i]
    right_fit = right_fit_list[i]

    # clear plt intance
    f = plt.figure(i)
    plt.clf()

    # Run image through the pipeline
    result = search_around_poly(bottom_half, left_fit, right_fit)
    plt.imshow(result)
    f.savefig(output_dir + os.path.basename(images[i]))


As we can see the fitting also works using the poly filter.

![Lanes Image](../output_images/lane_detection/poly_filter/test2.jpg) 

In [15]:
# testing the implementation in pipeline
from pipeline import find_lane_pixels_window, fit_poly, fit_around_poly
# select images between 0 and 7
fname = images[2]

# original
img = mpimg.imread(fname)
# plotimage(img, 0)

# undistorted
img_undist = pp.undistort_image(img)
# plotimage(img_undist, 1)

# bitmap with streigh guides
bitmap = pp.threshold(img_undist)

# eagle view with streigh guides
img_eagle_eye = pp.eagle_eye(bitmap)

x_half = int(img_eagle_eye.shape[1] / 2)
y_half = int(img_eagle_eye.shape[0] / 2)
bottom_half = img_eagle_eye[y_half:, :]

# computing poly_fit using fit_poly in pipeline
leftx, lefty, rightx, righty = find_lane_pixels_window(bottom_half, nwindows=10, margin=100, minpix=40)
left_fit, right_fit, left_fitx, right_fitx, ploty = fit_poly(bottom_half.shape, leftx, lefty, rightx, righty)

# Plot the polynomial lines onto the image
plt.figure(0)
plt.clf()
out_img = np.dstack((bottom_half, bottom_half, bottom_half))*255
plt.imshow(out_img)
plt.plot(left_fitx, ploty, color='yellow')
plt.plot(right_fitx, ploty, color='yellow')

# computing poly_fit using fit_around_poly in pipeline
left_fit, right_fit, left_fitx_around, right_fitx_around, ploty = fit_around_poly(bottom_half, left_fit, right_fit)

# Plot the polynomial lines onto the image
plt.figure(1)
plt.clf()
plt.imshow(out_img)
plt.plot(left_fitx_around, ploty, color='yellow')
plt.plot(right_fitx_around, ploty, color='yellow')